## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

In [6]:
from collections import defaultdict
import itertools

def Enumerator(start=0, initial=()):
    return defaultdict(itertools.count(start).__next__, initial)

In [7]:
import glob
import csv
from tqdm.auto import tqdm

id_enumerator = Enumerator()
row_dict = {}
left_id_set = set()
right_id_set = set()
rows_total = 1363 + 3226
clusters_total = 1300

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/Amazon-GoogleProducts/Amazon.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['name'] = row.pop('title')
            row['source'] = 'google'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/Amazon-GoogleProducts/GoogleProducts.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'amazon'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [8]:
true_pair_set = set()

with open(f'{home_dir}/Downloads/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[row['idAmazon']]
        id_right = id_enumerator[row['idGoogleBase']]
        true_pair_set.add(tuple(sorted([id_left, id_right])))

len(true_pair_set)

1300

In [9]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict

cluster_mapping, cluster_dict = id_pairs_to_cluster_mapping_and_dict(true_pair_set)
len(cluster_mapping)

2404

In [10]:
len(cluster_dict)

1105

In [11]:
# TODO: deal with this difference
# from entity_embed.data_utils.utils import cluster_dict_to_id_pairs

# assert len(true_pair_set - cluster_dict_to_id_pairs(cluster_dict)) == 0

In [12]:
cluster_attr = 'cluster_id'
max_cluster_id = max(cluster_mapping.values())

for row_id, row in tqdm(row_dict.items()):
    try:
        row[cluster_attr] = cluster_mapping[row_id]
    except KeyError:
        row[cluster_attr] = max_cluster_id
        max_cluster_id += 1

In [13]:
[row_dict[row_id] for row_id in next(iter(true_pair_set))]

[{'id': 938,
  'description': 'improve your typing skills today! typing instructor deluxe has a progressive design that has been developed for over 19 years. typing instructor deluxe can provide the right lessons tests strengthening exercises practice material and typing games for your skill level. you can even build your own personal typing plan to focus on specific areas you would like to improve. if you think learning has to be all hard work and no fun think again! for beginning to advanced typists kids to adults typing instructor deluxe will motivate you to improve your typing speed and accuracy using a travel theme and exciting typing challenges.educates entertains and motivates: choose from many typing plans or build your ownnavigate easily and choose your typing materialnew! dynamic learning methodsave your results and reports to track progresslearn voice-touch typing (dictation)3 unique travel themescolorful photos and musicten exciting games300+ magazine articles',
  'manufact

## Preprocess

In [14]:
attr_list = ['name', 'description', 'manufacturer', 'price']

In [15]:
import unidecode
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [16]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [17]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [18]:
attr_info_dict = {
    'name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'description': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [19]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

15:14:05 INFO:For attr='name', computing actual alphabet and max_str_len
15:14:05 INFO:For attr='name', using actual_max_str_len=26
15:14:05 INFO:For attr='description', computing actual alphabet and max_str_len
15:14:05 INFO:actual_max_str_len=29 must be pair to enable NN pooling. Updating to 30
15:14:05 INFO:For attr='description', using actual_max_str_len=30
15:14:05 INFO:For attr='manufacturer', computing actual alphabet and max_str_len
15:14:05 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
15:14:05 INFO:For attr='manufacturer', using actual_max_str_len=16
15:14:05 INFO:For attr='price', computing actual alphabet and max_str_len
15:14:05 INFO:For attr='price', using actual_max_str_len=14


{'name': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer=<function default_tokenizer at 0x7fbbbfc79e50>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=26, vocab=None),
 'description': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer=<function default_tokenizer at 0x7fbbbfc79e50>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', 

In [20]:
from entity_embed import LinkageDataModule

train_cluster_len = 200
valid_cluster_len = 200
datamodule = LinkageDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_numericalizer=row_numericalizer,
    pos_pair_batch_size=45,
    neg_pair_batch_size=1225,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=clusters_total - valid_cluster_len - train_cluster_len,
    only_plural_clusters=True,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    random_seed=random_seed
)

## Training

In [21]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    use_mask=True
)

In [22]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.3',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'amzn-googl'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

15:14:05 INFO:GPU available: True, used: True
15:14:05 INFO:TPU available: None, using: 0 TPU cores
15:14:05 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [23]:
trainer.fit(model, datamodule)

15:14:05 INFO:Train pair count: 292
15:14:05 INFO:Valid pair count: 265
15:14:05 INFO:Test pair count: 998
15:14:07 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 3.3 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params


1

In [24]:
model.blocker_net.get_signature_weights()

{'name': 0.2922545075416565,
 'description': 0.23582759499549866,
 'manufacturer': 0.23292307555675507,
 'price': 0.23899485170841217}

## Testing

In [25]:
trainer.test(ckpt_path='best')

15:15:08 INFO:Train pair count: 292
15:15:08 INFO:Valid pair count: 265
15:15:08 INFO:Test pair count: 998



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_at_0.3': 0.04982775590551181,
 'test_f1_at_0.5': 0.31115789473684213,
 'test_f1_at_0.7': 0.5966101694915255,
 'test_f1_at_0.9': 0.20062047569803518,
 'test_pair_entity_ratio_at_0.3': 20.596879063719115,
 'test_pair_entity_ratio_at_0.5': 2.5461638491547465,
 'test_pair_entity_ratio_at_0.7': 0.6085825747724317,
 'test_pair_entity_ratio_at_0.9': 0.08647594278283485,
 'test_precision_at_0.3': 0.025569796072984406,
 'test_precision_at_0.5': 0.1887129724208376,
 'test_precision_at_0.7': 0.5641025641025641,
 'test_precision_at_0.9': 0.7293233082706767,
 'test_recall_at_0.3': 0.9712230215827338,
 'test_recall_at_0.5': 0.8860911270983214,
 'test_recall_at_0.7': 0.6330935251798561,
 'test_recall_at_0.9': 0.11630695443645084}
--------------------------------------------------------------------------------


[{'test_precision_at_0.3': 0.025569796072984406,
  'test_recall_at_0.3': 0.9712230215827338,
  'test_f1_at_0.3': 0.04982775590551181,
  'test_pair_entity_ratio_at_0.3': 20.596879063719115,
  'test_precision_at_0.5': 0.1887129724208376,
  'test_recall_at_0.5': 0.8860911270983214,
  'test_f1_at_0.5': 0.31115789473684213,
  'test_pair_entity_ratio_at_0.5': 2.5461638491547465,
  'test_precision_at_0.7': 0.5641025641025641,
  'test_recall_at_0.7': 0.6330935251798561,
  'test_f1_at_0.7': 0.5966101694915255,
  'test_pair_entity_ratio_at_0.7': 0.6085825747724317,
  'test_precision_at_0.9': 0.7293233082706767,
  'test_recall_at_0.9': 0.11630695443645084,
  'test_f1_at_0.9': 0.20062047569803518,
  'test_pair_entity_ratio_at_0.9': 0.08647594278283485}]

## Testing manually 

In [26]:
# Only call this if test above wasn't run
# datamodule.setup(stage='test')

In [27]:
test_row_dict = datamodule.test_row_dict
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [28]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [29]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [30]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 391 ms, sys: 8.18 ms, total: 400 ms
Wall time: 57.5 ms


In [31]:
%%time

sim_threshold = 0.3
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 598 ms, sys: 24.1 ms, total: 623 ms
Wall time: 102 ms


In [32]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

20.596879063719115

In [33]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.025569796072984406, 0.9712230215827338)

In [34]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

30868

In [35]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

24

In [36]:
cos_similarity = lambda a, b: np.dot(a, b)

In [37]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.27086073,
 {'id': 352,
  'description': 'sony media software vegas 6 + dvd',
  'manufacturer': 'sony pictures home entertainment',
  'price': '899 . 99',
  'name': 'pc vegas dvd',
  'source': 'google',
  'cluster_id': 352},
 {'id': 4234,
  'name': 'vegas + dvd sony',
  'description': 'key features : three features in one professional dvds ...',
  'manufacturer': '',
  'price': '899 . 99',
  'source': 'amazon',
  'cluster_id': 352})

(0.28651214,
 {'id': 1183,
  'description': 'create the perfect pet for your sims to train play with and love throughout a lifetime . pick everything from their paws to their personality . choose from dozens of',
  'manufacturer': 'aspyr media',
  'price': '34 . 99',
  'name': 'sims 2 pets expansion pack',
  'source': 'google',
  'cluster_id': 1183},
 {'id': 2255,
  'name': 'sims 2 pets for mac',
  'description': 'system requirements : requires the full version of the sims 2 for mac os x to play operating system : mac os x 10 . 3 . 9 or later',
  'manufacturer': '',
  'price': '34 . 99',
  'source': 'amazon',
  'cluster_id': 1183})

(0.005087917,
 {'id': 125,
  'description': '',
  'manufacturer': 'compaq computer',
  'price': '0',
  'name': 'compaq comp . rapid deployment pk - flexible lic kit ( 302127 - b21 )',
  'source': 'google',
  'cluster_id': 125},
 {'id': 3231,
  'name': 'hewlett packard 302127 - b21 prol essentials rdp v1 . x 1u flex lic min qty 5',
  'description': 'prol essentials rdp v1 . x 1u flex lic min qty 5',
  'manufacturer': '',
  'price': '119 . 73',
  'source': 'amazon',
  'cluster_id': 125})

(0.007058701,
 {'id': 845,
  'description': "iplaymusic ' s beginner guitar lessons is the first guitar learning program optimized for the mac and video ipod . the software takes advantage of ilife applications resident on today",
  'manufacturer': 'iplaymusic',
  'price': '49 . 99',
  'name': 'iplaymusic beginner guitar lessons for the mac and ipod',
  'source': 'google',
  'cluster_id': 845},
 {'id': 2324,
  'name': "wingnuts 2 : raina ' s revenge",
  'description': 'system requirements : mac os x 10 . 4 + g4 / g5 / intel 800 + mhz cpu 512 mb ram 32 mb video card 950 mb hard drive',
  'manufacturer': '',
  'price': '28 . 99',
  'source': 'amazon',
  'cluster_id': 845})

(0.13004163,
 {'id': 870,
  'description': '- marketing information : tinyterm provides accurate emulation and total flexibility . now you can use your desktop pc to access all your legacy data and applications . with tinyterm',
  'manufacturer': 'century software',
  'price': '219 . 63',
  'name': 'tiny term emulator v4 . 3x',
  'source': 'google',
  'cluster_id': 870},
 {'id': 3851,
  'name': 'century software tt - 1 - century tinyterm v . 4 . 3x - emulation - 1 user ( s ) - english french german spanish italian polish portuguese -',
  'description': 'century software tt - 1 : tinyterm provides accurate emulation and total flexibility . now you can use your desktop pc to access all your legacy data and applications .',
  'manufacturer': '',
  'price': '120 . 97',
  'source': 'amazon',
  'cluster_id': 870})

(0.2925953,
 {'id': 641,
  'description': "sims 2 : nightlife takes your sim into the night . explore all of your favorite after - dark activities as you discover your sims ' love lives or have",
  'manufacturer': 'aspyr media',
  'price': '34 . 99',
  'name': 'sims 2 nightlife expansion pack',
  'source': 'google',
  'cluster_id': 641},
 {'id': 1908,
  'name': 'aspyr media inc sims 2 nightlife',
  'description': "send your sims on an epic night out ! your sims are on the town hitting all the swanky hot spots . whether they ' re dancing until dawn romancing",
  'manufacturer': '',
  'price': '33 . 21',
  'source': 'amazon',
  'cluster_id': 641})

(0.2904828,
 {'id': 43,
  'description': '',
  'manufacturer': 'imsi design',
  'price': '0',
  'name': 'clip art and more 250 000 ( jewel case )',
  'source': 'google',
  'cluster_id': 43},
 {'id': 3429,
  'name': 'clipart & more 250000 ( jc )',
  'description': 'top - quality royalty - free images use these royalty - free images in any project advertising brochures newsletters postcards craft projects labels posters with any print publishing program !',
  'manufacturer': '',
  'price': '9 . 9',
  'source': 'amazon',
  'cluster_id': 43})

(0.25740534,
 {'id': 199,
  'description': 'upg rumba unix - hp edition level a 1 - 4 units',
  'manufacturer': 'netmanage',
  'price': '0',
  'name': 'netmanage upg rumba unix - hp edition level ( 902453 - 007 - c )',
  'source': 'google',
  'cluster_id': 199},
 {'id': 1428,
  'name': '902453 - 007 - b rumba unix - hp edition - upgrade license - 1 user - volume - level b ( 5 - 49 ) - c',
  'description': 'netmanage 902453 - 007 - b : usually ships in 24 hours : : the rumba product family provides users secure access to mission - critical applications and data residing',
  'manufacturer': '',
  'price': '147 . 73',
  'source': 'amazon',
  'cluster_id': 199})

(0.13024785,
 {'id': 799,
  'description': '- marketing information : hp digital sending software 4 . 0 improves core business processes . digital sending streamlines critical business document handling and integrates with existing it infrastructures to',
  'manufacturer': 'hewlett packard ( consumables )',
  'price': '630 . 36',
  'name': 'hp dss software - ( v . 4 . 0 ) - complete package ( t1936aa ua0 )',
  'source': 'google',
  'cluster_id': 799},
 {'id': 2167,
  'name': 'hewlett packard t1936aa ua0 - hp digital sending software 4 . 0 - complete product - document management - 1 device ( s ) - complete product - standard -',
  'description': 'hewlett packard t1936aa ua0 : the hp mfp digital sending software ( hp dss ) is a software application that enables an hp multifunction peripheral ( mfp ) to send',
  'manufacturer': '',
  'price': '342 . 97',
  'source': 'amazon',
  'cluster_id': 799})

(-0.08183968,
 {'id': 905,
  'description': "efrontier ' s anime studio pro 5 is a complete animation program for creating movies cartoons anime or cut animations . create your own desktop animated shorts in the style",
  'manufacturer': 'efrontier',
  'price': '199 . 99',
  'name': 'efrontier anime studio pro ( win / mac )',
  'source': 'google',
  'cluster_id': 905},
 {'id': 1529,
  'name': 'allume anime studio pro',
  'description': 'mac os x v10 . 3 . 9 + ( universal binary ) ( mac os x v10 . 4 + recommended ) powerpc g5 / g4 processor 500mhz or',
  'manufacturer': '',
  'price': '179 . 99',
  'source': 'amazon',
  'cluster_id': 905})